### Wczytanie danych

In [82]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import re
import string
from stempel import StempelStemmer



In [263]:
#dodać taki df do pliku:
# kiid_text = {'ID_KIID':[1,2,3,...],
#    'TEKST':[cały string z tekstem z pdf nr 1, nr2, nr3, ...]} (w tekście nie ma już \n )!
#    df_kiid_text = pd.DataFrame(data=kiid_text)
# stopword.txt pobrany
df_kiid_text = pd.read_csv('data.csv')#zamienić na ścieżke do pliku




### Czyszczenie

In [389]:


def clean_text_round1(text):
    #Tekst do małych liter, usunięcie znaków interpunkcyjnych, słów zawierających litery
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

def clean_text_round2(text):
    #Usunięcie stop wyrazów
    with open('stopwords.txt') as f:
        
        stopwords = []
        for line in f:
            line=line.strip()
            stopwords.append(line)
        
        for stopword in stopwords:
            text = re.sub(r"\b%s\b" %stopword, '', text)
    return text

round2 = lambda x: clean_text_round2(x)


In [402]:
data_clean = pd.DataFrame(df_kiid_text.TEKST.apply(round1))
data_clean2 = pd.DataFrame(data_clean.TEKST.apply(round2))
data_clean['ID_KIID']=df_kiid_text.ID_KIID
data_clean2['ID_KIID']=df_kiid_text.ID_KIID

### Tabela KIID_BAGOFWORDS_S

In [415]:

cv = CountVectorizer()
data_cv = cv.fit_transform(data_clean.TEKST)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names()) #rozdzielenie i zliczenie słów
data_dtm.index = data_clean.index
data_dtm['ID_KIID']=df_kiid_text.ID_KIID

df_KIID_BAGOFWORDS_S= data_dtm.melt(id_vars=['ID_KIID'])
df_KIID_BAGOFWORDS_S.columns = ["ID_KIID","SLOWO","LICZBA_WYSTAPIEN"]
df_KIID_BAGOFWORDS_S = df_KIID_BAGOFWORDS_S[df_KIID_BAGOFWORDS_S.LICZBA_WYSTAPIEN != 0]  #usunięcie wierszy w których liczność jest zero
df_KIID_BAGOFWORDS_S['ID_ZESPOLU'] = 13  #dodanie id zespolu
df_KIID_BAGOFWORDS_S = df_KIID_BAGOFWORDS_S[['ID_KIID','ID_ZESPOLU','SLOWO','LICZBA_WYSTAPIEN']] #zamiana kolejności kolumn
df_KIID_BAGOFWORDS_S = df_KIID_BAGOFWORDS_S.reset_index()

#df_KIID_BAGOFWORDS_S to ostateczny df dla KIID_BAGOFWORDS_S



### Tabela KIID_BAGOFWORDS_N

In [67]:
stemmer = StempelStemmer.polimorf()

Loading: 100%|███████████████| 11368252/11368252 [00:18<00:00, 600044.26bytes/s]


In [420]:
cv = CountVectorizer()
data_cv = cv.fit_transform(data_clean2.TEKST)  #teraz na tekście wyczyszczonym ze stop słów
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names()) #rozdzielenie i zliczenie słów
data_dtm.index = data_clean2.index


list_stemmer = []             #normalizacja końcówek
for word in data_dtm.columns :
    list_stemmer.append(stemmer.stem(word))
    
data_dtm.columns = list_stemmer
data_dtm= data_dtm.groupby(by=data_dtm.columns, axis=1).sum()
data_dtm['ID_KIID']=df_kiid_text.ID_KIID
df_KIID_BAGOFWORDS_N= data_dtm.melt(id_vars=['ID_KIID'])
df_KIID_BAGOFWORDS_N.columns = ["ID_KIID","SLOWO","LICZBA_WYSTAPIEN"]
df_KIID_BAGOFWORDS_N['ID_ZESPOLU'] = 13   #dodanie id zespolu
df_KIID_BAGOFWORDS_N = df_KIID_BAGOFWORDS_N[['ID_KIID','ID_ZESPOLU','SLOWO','LICZBA_WYSTAPIEN']] #zamiana kolejności kolumn
df_KIID_BAGOFWORDS_N = df_KIID_BAGOFWORDS_N.reset_index()


#df_KIID_BAGOFWORDS_N to ostateczny df dla KIID_BAGOFWORDS_N



### Tabela KIID_WYRAZENIA

In [426]:
w_wyrazenia = ["Kluczowe informacje dla inwestorów",
"Niniejszy dokument zawiera kluczowe informacje dla inwestorów dotyczące tego funduszu. Nie są to materiały marketingowe. Dostarczenie tych informacji jest wymogiem prawnym mającym na celu ułatwienie zrozumienia charakteru i ryzyka związanego z inwestowaniem w ten fundusz. Przeczytanie niniejszego dokumentu jest zalecane inwestorowi, aby mógł on podjąć świadomą decyzję inwestycyjną.",
"Niniejsze kluczowe informacje dla inwestorów są aktualne na dzień",
 "Fundusz otrzymał zezwolenie na prowadzenie działalności w",
"Zalecenie: niniejszy fundusz może nie być odpowiedni dla inwestorów, którzy planują wycofać swoje środki w ciągu",
"może zostać pociągnięta do odpowiedzialności za każde oświadczenie zawarte w niniejszym dokumencie, które wprowadza w błąd, jest niezgodne ze stanem faktycznym lub niespójne z odpowiednimi częściami prospektu emisyjnego UCITS.",
"Opłaty jednorazowe pobierane przed lub po dokonaniu inwestycji",
 "Opłata za subskrypcję",
"Opłata za umorzenie",
"Opłaty pobierane z funduszu w ciągu roku",
"Opłaty bieżące",
"Opłaty pobierane z funduszu w określonych warunkach szczególnych",
"Opłata za wyniki",
"Cele i polityka inwestycyjna",
"Profil ryzyka i zysku",
"Opłaty",
"Wyniki osiągnięte w przeszłości",
"Informacje praktyczne"]  #lista wyrażeń kluczowych

In [427]:
import string
import math
def check_expresion(x,y,z):  #funkcja sprawdza czy dane wyrażenie 'y', jest w tekście 'x'
    y = y.lower()            #z błędem 'z' (wpisany w systemie dziesiętnym np. błąd 5% to z=0.05), liczbę znaków na błąd zaokrąglam w górę od długości wyrażenia
    y_clear = re.sub('\w*\d\w*', '', y)
    y_clear = re.sub('[%s]' % re.escape(string.punctuation), '', y_clear)
    y_clear = re.sub(r'\s+', '', y_clear)

    x = x.lower()

    for i in range(0, len(x)-len(y)+1):

        sent = x[i:math.ceil(i+2*z*len(y)+len(y))]

        clear_sent = re.sub('\w*\d\w*', '', sent)
        clear_sent = re.sub('[%s]' % re.escape(string.punctuation), '', clear_sent)
        clear_sent = re.sub(r'\s+', '', clear_sent)

        diff = 0
        if (y_clear in clear_sent):
            n_y = 0
            n_s = 0
            while(n_y<len(y) and n_s<len(s)):
                ch_y = y[n_y]
                ch_s = sent[n_s]

                if ch_y == ch_s:   #jak takie same to idę dalej
                    n_y=n_y+1
                    n_s=n_s+1
                else :   #teraz przypadki co się nie zgadza i naliczam różniące się znaki
                    diff=diff+1
                    if ch_y.isalpha() and ch_s.isalpha(): #w zależności od przypadku inaczej zmieniam iteratory
                        n_y=n_y+1                         
                        n_s=n_s+1
                    elif ch_s.isspace() and (ch_y in string.punctuation):
                        n_y=n_y+1        #przykładowo jeżeli ktoś zamiast kropki dał w dokumencie spację 
                        n_s=n_s+1        #przechodzę do następnego znaku w obu iteratorach

                    elif ch_s.isspace(): #przykładowo jeśli mam dodatkową spację 
                        n_y=n_y          #to chcę przejść dalej znak w dokumencie, a zostawić w wyrażeniu
                        n_s=n_s+1

                    elif (ch_s in string.punctuation) and (ch_y in string.punctuation) :
                        n_y=n_y+1
                        n_s=n_s+1

                    elif ch_s in string.punctuation:
                        n_y=n_y
                        n_s=n_s+1
                    elif ch_y in string.punctuation and ch_s.isalpha():
                        n_y=n_y+1
                        n_s=n_s
                    else:
                        n_y=n_y+1
                        n_s=n_s+1
            if diff<=math.ceil(len(y)*z):
                return True

        
    return False
            
                                      
                                      

In [428]:
df_KIID_WYRAZENIA = pd.DataFrame(columns=w_wyrazenia) 
df_KIID_WYRAZENIA['ID_KIID'] = df_kiid_text.ID_KIID
for id in df_kiid_text.ID_KIID:  #sprawdzanie czy w danym dokumencie wystąpiło dane wyrażenie i dodanie do tabeli wartości boolean
    for w in w_wyrazenia:
          df_KIID_WYRAZENIA.loc[df_KIID_WYRAZENIA.ID_KIID==id,w] = check_expresion(df_kiid_text[df_kiid_text.ID_KIID==id].TEKST.values[0],w,0.05)


In [429]:
df_KIID_WYRAZENIA = df_KIID_WYRAZENIA.melt(id_vars=['ID_KIID'])    #przekształcenie tabeli
df_KIID_WYRAZENIA.columns = ["ID_KIID","WYRAZENIE","FLAGA_WYSTAPIENIA"]
df_KIID_WYRAZENIA['ID_ZESPOLU'] = 13   #dodanie id zespolu
df_KIID_WYRAZENIA = df_KIID_WYRAZENIA[['ID_KIID','ID_ZESPOLU','WYRAZENIE','FLAGA_WYSTAPIENIA']] #zamiana kolejności kolumn
#df_KIID_WYRAZENIA to ostateczny df dla KIID_WYRAZENIA


### Zapisanie do csv

In [ ]:
df_KIID_BAGOFWORDS_S.to_csv('mISIe_KIID_BAGOFWORDS_S.csv') #poprawić zapis
df_KIID_BAGOFWORDS_N.to_csv('mISIe_KIID_BAGOFWORDS_N.csv') #poprawić zapis
df_KIID_WYRAZENIA.to_csv('mISIe_KIID_WYRAZENIA.csv') #poprawić zapis


